# Scraper Testing Notebook

**Purpose**: Test ONLY the web scrapers - no sentiment analysis, minimal database

## What We Test
1. ✅ Gemini AI Search Term Generator
2. ✅ IMDb Rating Scraper  
3. ✅ IMDb Review Scraper
4. ⚠️  Reddit Scraper (needs API keys)
5. ⚠️  Twitter Scraper (optional)

## What We DON'T Test
- ❌ Sentiment analysis (NLP team)
- ❌ Review weighting (NLP team)
- ❌ Complex SQL operations
- ❌ Recommendation models

## Step 1: Setup & Imports

In [ ]:
import sys
from pathlib import Path
import os
from dotenv import load_dotenv
import json
from pprint import pprint

# Add src to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

# Load environment variables
load_dotenv(project_root / '.env')

print("✅ Setup complete!")
print(f"📁 Project root: {project_root}")

## Step 2: Test Gemini AI Search Term Generator

In [ ]:
from scrapers.gemini_search import GeminiSearchTermGenerator

# Check API key
gemini_api_key = os.getenv('GEMINI_API_KEY')
if not gemini_api_key:
    print("❌ GEMINI_API_KEY not found in .env")
else:
    print(f"✅ Gemini API key loaded: {gemini_api_key[:20]}...")

# Initialize generator
gemini = GeminiSearchTermGenerator()
print("✅ Gemini generator initialized!")

In [ ]:
# Test: Generate search terms for "Inception"
print("🎬 Testing Gemini with 'Inception (2010)'...\n")

search_terms = gemini.generate_search_terms(
    title="Inception",
    year=2010,
    genres=["Action", "Sci-Fi", "Thriller"],
    overview="A thief who steals corporate secrets through dream-sharing technology."
)

print("📊 Result type:", type(search_terms))
print("📊 Keys:", list(search_terms.keys()) if search_terms else "None")
print("\n" + "="*80)

if search_terms:
    for platform, terms in search_terms.items():
        print(f"\n{platform.upper()}:")
        for term in terms[:3]:  # Show first 3 terms
            print(f"  • {term}")
        if len(terms) > 3:
            print(f"  ... and {len(terms)-3} more")
else:
    print("⚠️  No search terms generated")

## Step 3: Test IMDb Rating Scraper

In [ ]:
from scrapers.imdb_scraper import IMDbScraper

# Initialize scraper
imdb = IMDbScraper(rate_limit=2.0)
print("✅ IMDb scraper initialized!")
print("⏱️  Rate limit: 2 seconds between requests")

In [ ]:
# Test: Scrape rating for "Inception"
print("🎬 Testing IMDb rating scraper with 'Inception (2010)'...\n")

rating_data = imdb.scrape_movie_rating(
    title="Inception",
    year=2010
)

print("📊 Result type:", type(rating_data))
print("\n" + "="*80)

if rating_data:
    print("\n✅ RATING DATA:")
    print(f"  Rating: {rating_data.get('rating')}/10")
    print(f"  Votes: {rating_data.get('vote_count'):,}")
    print(f"  IMDb ID: {rating_data.get('imdb_id')}")
else:
    print("⚠️  No rating data found")

## Step 4: Test IMDb Review Scraper

In [ ]:
# Test: Scrape reviews for "Zootopia"
print("🎬 Testing IMDb review scraper with 'Zootopia (2016)'...\n")
print("⏱️  This will take ~20 seconds (10 reviews with 2-second rate limit)\n")

reviews = imdb.scrape_movie_reviews(
    title="Zootopia",
    year=2016,
    max_reviews=10
)

print("\n" + "="*80)
print(f"\n✅ Scraped {len(reviews)} reviews\n")

if reviews:
    print("📊 SAMPLE REVIEW (first one):")
    sample = reviews[0]
    print(f"\n  Source: {sample.get('source')}")
    print(f"  Rating: {sample.get('rating')}/10")
    print(f"  Author: {sample.get('author')}")
    print(f"  Helpful votes: {sample.get('helpful_count')}")
    print(f"  Text preview: {sample.get('text', '')[:200]}...")
    print(f"\n  Full structure:")
    pprint({k: v for k, v in sample.items() if k != 'text'}, indent=4)
else:
    print("⚠️  No reviews found")

## Step 5: Test Reddit Scraper (Optional)

In [ ]:
# Check if Reddit API keys are available
reddit_client_id = os.getenv('REDDIT_CLIENT_ID')
reddit_client_secret = os.getenv('REDDIT_CLIENT_SECRET')

if reddit_client_id and reddit_client_secret:
    print("✅ Reddit API keys found!")
    from scrapers.reddit_scraper import RedditScraper
    
    reddit = RedditScraper()
    print("✅ Reddit scraper initialized!")
    
    # Test search
    print("\n🔍 Searching Reddit for 'Inception movie discussion'...\n")
    reddit_posts = reddit.search_posts(
        search_terms=["Inception movie discussion"],
        max_results_per_term=5
    )
    
    print(f"✅ Found {len(reddit_posts)} Reddit posts")
    if reddit_posts:
        print("\n📊 SAMPLE POST:")
        sample = reddit_posts[0]
        print(f"  Subreddit: {sample.get('subreddit')}")
        print(f"  Score: {sample.get('score')}")
        print(f"  Text: {sample.get('text', '')[:200]}...")
else:
    print("⚠️  Reddit API keys not found in .env")
    print("   To test Reddit scraper, add:")
    print("   REDDIT_CLIENT_ID=your_id")
    print("   REDDIT_CLIENT_SECRET=your_secret")

## Step 6: Test Twitter Scraper (Optional)

In [ ]:
print("⚠️  Twitter scraper uses snscrape (no API key needed)")
print("   However, Twitter/X has been restricting scraping recently.")
print("   Test at your own risk - may not work reliably.\n")

# Uncomment to test:
# from scrapers.twitter_scraper import TwitterScraper
# twitter = TwitterScraper()
# tweets = twitter.search_tweets(
#     search_terms=["#Inception movie"],
#     max_tweets_per_term=5
# )
# print(f"Found {len(tweets)} tweets")

## Summary: Scraper Test Results

In [ ]:
print("="*80)
print("SCRAPER TEST SUMMARY")
print("="*80)
print()
print("✅ Gemini AI: Generates search terms (dict with platform keys)")
print("✅ IMDb Rating: Returns rating, vote count, IMDb ID")
print("✅ IMDb Reviews: Returns list of review dictionaries")
print("⚠️  Reddit: Needs API keys in .env")
print("⚠️  Twitter: May not work due to platform restrictions")
print()
print("="*80)
print("NEXT STEPS")
print("="*80)
print()
print("1. Scrapers are working independently ✅")
print("2. Data structures are correct ✅")
print("3. Ready to integrate with database")
print("4. NLP team can process scraped reviews")
print("5. Recommendation team can use rated movies")